Download the images from the google drive for feature detection.

In [ ]:
!pip install -q gdown

In [ ]:
!gdown --id 1BEK4DnnkEOl8dKl8lwSjbeVQCFSJgfdg -O checkerboard_img.png

Let's start with detecting corners on images. To verify that the corner detector works perfectly, we start with a checkerboard pattern and make sure that corners are detected on the four corners of each grid on the checkerboard.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import cv2

checkerboard_img = imageio.imread("/content/checkerboard_img.png")
print(checkerboard_img.shape)

plt.figure(1)
plt.imshow(checkerboard_img, cmap='gray')
plt.title("Checkerboard Image")
plt.axis("off")

Notice that inside the `get_corners` function below, we adopt a derivative of Gaussian to find the derivatives of the image in x and y directions, since a corner is a point on the image where the gradient is significant in all directions. A Gaussian convolution operation is applied subsequently to remove noise (as we have done in the image noise removal practice). The corner detector should find corners at the intersections of the grids of our checkerboard pattern.

In [ ]:
def gaussian_derivative_kernels(ksize=5, sigma=1.0):
    ax = np.arange(-(ksize // 2), ksize // 2 + 1)
    xx, yy = np.meshgrid(ax, ax)
    g = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    gx = -xx / (sigma**2) * g
    gy = -yy / (sigma**2) * g
    gx /= np.sum(np.abs(gx))
    gy /= np.sum(np.abs(gy))
    return gx, gy

def get_corners(img, sobel_size=5, gauss_ksize=5, gauss_sigma=1):

  gx, gy = gaussian_derivative_kernels(ksize=gauss_ksize, sigma=gauss_sigma)

  # Compute image gradients in both x and y directions
  Ix = cv2.filter2D(img.astype(np.float32), -1, gx)
  Iy = cv2.filter2D(img.astype(np.float32), -1, gy)

  # Compute products of derivatives
  Ixx = Ix * Ix
  Iyy = Iy * Iy
  Ixy = Ix * Iy

  # Apply Gaussian filter to smooth the products
  Ixx = cv2.GaussianBlur(Ixx, (gauss_ksize, gauss_ksize), gauss_sigma)
  Iyy = cv2.GaussianBlur(Iyy, (gauss_ksize, gauss_ksize), gauss_sigma)
  Ixy = cv2.GaussianBlur(Ixy, (gauss_ksize, gauss_ksize), gauss_sigma)

  # calculate the Harris response
  k = 0.04
  detM = Ixx * Iyy - Ixy * Ixy
  traceM = Ixx + Iyy
  R = detM - k * (traceM ** 2)

  # Normalize response
  R_norm = cv2.normalize(R, None, 0, 255, cv2.NORM_MINMAX)

  # Non-Maximum Suppression (NMS)
  thresh = 0.01 * R.max()

  # Binary map of candidate corners
  candidate_mask = (R > thresh)

  # Keep only local maxima
  kernel = np.ones((3, 3), np.uint8)
  local_max = (R == cv2.dilate(R, kernel))
  b_corner_points = np.logical_and(candidate_mask, local_max)
  ys, xs = np.where(b_corner_points)
  return ys, xs


checkerboard_img = np.float32(checkerboard_img)
corners_y, corners_x = get_corners(checkerboard_img)

# Display results
plt.figure(figsize=(6, 6))
plt.imshow(np.uint8(checkerboard_img), cmap='gray')
plt.scatter(corners_x, corners_y, s=10, c='red', marker='.', label='Corners')
plt.title("Corners on the Checkerboard Image")
plt.axis("off")
plt.legend()
plt.show()



Now that the corner detector works perfectly, let's move on to extract corners on a real image captured by a Arducam camera.

In [ ]:
!gdown --id 132QiEWBa8SqbQC4Y1AAAYwEFUho7zX0E -O chiangheng.png

In [ ]:
funny_chiangheng_img = imageio.imread("/content/chiangheng.png")

plt.figure(2)
plt.imshow(funny_chiangheng_img, cmap='gray')
plt.title("Funny Chiang-Heng")
plt.axis("off")

In [ ]:
gray_img = cv2.cvtColor(funny_chiangheng_img, cv2.COLOR_BGR2GRAY)

corners_y, corners_x = get_corners(np.float32(gray_img), 3, 17, 5)
print(f'Number of corners = {len(corners_y)}')

# Display results
plt.figure(figsize=(6, 6))
plt.imshow(np.uint8(gray_img), cmap='gray')
plt.scatter(corners_x, corners_y, s=20, c='red', marker='.', label='Corners')
plt.title("Corners on Chiang-Heng")
plt.axis("off")
plt.legend()
plt.show()

As opposed to corners, SIFT is a robust feature detection method. A SIFT feature, or a "keypoint", is typically anchored with SIFT descriptors represented by a 128 dimensional vector.

In [ ]:
# Create SIFT constructor
sift = cv2.SIFT_create()

# Detect SIFT keypoints and descriptors
keypoints, descriptors = sift.detectAndCompute(funny_chiangheng_img, None)

print(f"Number of detected SIFT keypoints = {len(keypoints)}")
print(f"Descriptor shape of SIFT features = {descriptors.shape}")

for kp in keypoints[:10]:  # show first 10
    print(f"Point: {kp.pt}, Size: {kp.size}, Angle: {kp.angle}")

# Superimpose keypoints on the image
img_kp = cv2.drawKeypoints(
    funny_chiangheng_img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))
plt.title("SIFT Keypoints on Funny Chiang-Heng")
plt.axis("off")
plt.show()